In [1]:
import os
from pydicom import dcmread
from pydicom.data import get_testdata_file
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

In [2]:
def getAllMRI(dirName):
    mri_list = os.listdir(dirName)
    all_MRI = []
    for file in mri_list:
        full_path = os.path.join(dirName, file)
        if os.path.isdir(full_path):
            all_MRI = all_MRI + getAllMRI(full_path)
        else:
            all_MRI.append(full_path)
                
    return all_MRI

mri_files = getAllMRI('PROSTATE-MRI/MIP-PROSTATE-01-0001/04-30-2006-MRI Prostate ERC-40831/701.000000-T2 TSE ax hi-65190')

In [3]:
ds = dcmread(mri_files[0])
#px.imshow(ds.pixel_array, color_continuous_scale='gray')

In [4]:
volume = []
for mri in mri_files:
    mri_img = dcmread(mri)
    volume.append(mri_img.pixel_array)
volume = np.array(volume)

In [5]:
r, c = volume[0].shape

nb_frames = len(mri_files)

fig = go.Figure(
    frames = [
        go.Frame(
            data = go.Surface(
                z = (6.7 - k * 0.1) * np.ones((r, c)),
                surfacecolor=np.flipud(volume[len(mri_files) - 1 - k]),
                cmin=0, cmax=200
            ),
            name = str(k)
        )
        for k in range(nb_frames)
    ]
)
fig.add_trace(
    go.Surface(
        z = 6.7 * np.ones((r, c)),
        surfacecolor = np.flipud(volume[17]),
        colorscale = 'Gray',
        cmin = 0, cmax = 200,
        colorbar = dict(thickness = 20, ticklen = 4)
    )
)
pass

In [ ]:
def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=1024,
         height=1280,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

#fig.show()